In [11]:
!pip install pyvis
import random
from pyvis.network import Network
import pyvis


Defaulting to user installation because normal site-packages is not writeable


In [16]:
def create_network():
    network=Network(notebook=True, cdn_resources="remote")
  
    for i in range(12):
    
            network.add_node(i, label=f"PC{i}")
    color_switch ="#808080"  
    for i in range(6):
            network.add_node(i+12, label=f"S{i}",color=color_switch)
    color_router = "#000000"
    for i in range(5):
            network.add_node(i+18, label=f"R{i}",color=color_router)
    edges = []
    i=0
    j=0
    while i<12:
        for j in range(6):
            for k in range(2):
                    x=j+12
                    edge_wt = (x,i,3) 
                    edges.append(edge_wt)
                    network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
                    i=i+1
        x=0
        l=12
   
        for m in range(2):
       
                 for n in range(3):
               
                    edge_wt = (l,m+18,3) 
                    edges.append(edge_wt)
                    network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
                    l=l+1
        for m in range(3):
                edge_wt = (20,m+14, 3)
                edges.append(edge_wt)
                network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")

        edge_wt = (18,19, 3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
        
        edge_wt = (18,20, 3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")

        edge_wt = (19,20, 3)
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")

        edge_wt = (21,22, 3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
    

        edge_wt = (20,22, 3)
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
   

        edge_wt = (19,22,3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")

        edge_wt = (18,21, 3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")

        edge_wt = (19,21, 3) 
        edges.append(edge_wt)
        network.add_edge(edge_wt[0], edge_wt[1], label=str(edge_wt[2]), title=f"Weight: {edge_wt[2]}")
  
  
        
        return edges

In [17]:
import numpy as np
def create_adjacency_matrix(edges, num_nodes):
    # Initialize the matrix with 'inf' for no connection
    adjacency_matrix = np.full((num_nodes, num_nodes), float('inf'))

    # Set the diagonal to 0 (distance from a node to itself)
    np.fill_diagonal(adjacency_matrix, 0)

    # Populate the matrix with the edges
    for u, v, weight in edges:
        adjacency_matrix[u][v] = weight
        adjacency_matrix[v][u] = weight  # Assuming the graph is undirected

    return adjacency_matrix

In [18]:
def adjacency_matrix_to_edge_list(adjacency_matrix):
    num_nodes = len(adjacency_matrix)
    edge_list = []

    # Iterate through the adjacency matrix
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):  # To avoid duplicating edges in an undirected graph
            weight = adjacency_matrix[i][j]
            if weight != float('inf'):  # Ignore edges with no connection (infinity weight)
                edge_list.append((i, j, weight))

    return edge_list


In [19]:
import heapq

def dijkstra(adjacency_matrix, start, end, num_nodes):
    # Initialize distances and predecessors
    distances = [float('inf')] * num_nodes
    distances[start] = 0
    predecessors = [None] * num_nodes
    priority_queue = [(0, start)]  # (distance, node)

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # If the current node is the end node, we can stop early
        if current_node == end:
            break

        # Skip processing if a shorter path to current_node has already been found
        if current_distance > distances[current_node]:
            continue

        # Check neighbors and update distances using adjacency matrix
        for neighbor in range(num_nodes):
            edge_weight = adjacency_matrix[current_node][neighbor]

            if edge_weight != float('inf'):  # If there is a valid edge
                if distances[current_node] + edge_weight < distances[neighbor]:
                    distances[neighbor] = distances[current_node] + edge_weight
                    predecessors[neighbor] = current_node
                    heapq.heappush(priority_queue, (distances[neighbor], neighbor))

    # Reconstruct the shortest path
    path = []
    node = end
    while node is not None:
        path.insert(0, node)
        node = predecessors[node]

    # If the distance to the end is still infinity, there's no path
    if distances[end] == float('inf'):
        return float('inf'), []

    return distances[end], path


In [20]:
def bellman_ford(adjacency_matrix, num_nodes, start, end):
    # Initialize distances and predecessors
    distances = [float('inf')] * num_nodes
    distances[start] = 0
    predecessors = [None] * num_nodes

    # Relax all edges |V| - 1 times
    for _ in range(num_nodes - 1):
        for u in range(num_nodes):
            for v in range(num_nodes):
                weight = adjacency_matrix[u][v]
                if weight != float('inf'):  # If there is an edge
                    if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                        distances[v] = distances[u] + weight
                        predecessors[v] = u

    # Check for negative-weight cycles
    for u in range(num_nodes):
        for v in range(num_nodes):
            weight = adjacency_matrix[u][v]
            if weight != float('inf'):  # If there is an edge
                if distances[u] != float('inf') and distances[u] + weight < distances[v]:
                    print("Graph contains a negative-weight cycle.")
                    return float('inf'), []

    # Reconstruct the path from start to end
    path = []
    node = end
    while node is not None:
        path.insert(0, node)
        node = predecessors[node]

    # If the distance to the end is still infinity, there's no path
    if distances[end] == float('inf'):
        return float('inf'), []

    return distances[end], path


In [21]:
def apply_congestion_1(adjacency_matrix, n1, n2, n3):
    for u in range(23):
        if u != n1 and adjacency_matrix[n1][u] != float('inf'):  # Check if there's an edge
            if u == n2 or u == n3:
                adjacency_matrix[n1][u] += 30
                adjacency_matrix[u][n1] += 30
            else:
                adjacency_matrix[n1][u] += 20
                adjacency_matrix[u][n1] += 20
    return adjacency_matrix


In [22]:
def apply_congestion_2(adjacency_matrix, n1, n2, n3):
    for u in range(23):
        if u != n1 and u != n2 and adjacency_matrix[n2][u] != float('inf'):  # Check if there's an edge
            if u == n3:
                adjacency_matrix[n2][u] += 30
                adjacency_matrix[u][n2] += 30
            else:
                adjacency_matrix[n2][u] += 20
                adjacency_matrix[u][n2] += 20
    return adjacency_matrix


In [23]:
def apply_congestion_3(adjacency_matrix, n1, n2, n3):
    for u in range(23):
        if u != n1 and u != n2 and u != n3 and adjacency_matrix[n3][u] != float('inf'):  # Check if there's an edge
            adjacency_matrix[n3][u] += 20
            adjacency_matrix[u][n3] += 20
    return adjacency_matrix


In [24]:
from pyvis.network import Network

def main():
    # Number of nodes in the graph
    num_nodes = 23
    start_node = 3
    end_node = 9
    n1, n2, n3 = 19, 20, 18  # Nodes for congestion scenarios

    # Create the network and generate the edge list
    edges = create_network()  # Create the network and get the edges
    print("Generated Edges:", edges)

    # Convert edge list to adjacency matrix
    adjacency_matrix = create_adjacency_matrix(edges, num_nodes)
    print("Adjacency Matrix:")
    print(adjacency_matrix)

    # Running Dijkstra's Algorithm before congestion
    print("Before any congestion:")
    print(f"Running Dijkstra's Algorithm from {start_node} to {end_node}")
    distance, path = dijkstra(adjacency_matrix, start_node, end_node, num_nodes)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    # Run Bellman-Ford Algorithm after congestion scenario 1
    print(f"Running Bellman-Ford Algorithm from {start_node} to {end_node}")
    distance, path = bellman_ford(adjacency_matrix, num_nodes, start_node, end_node)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")
    
    # Apply congestion scenario 1 and run Dijkstra
    print("Applying Congestion Scenario 1...")
    adjacency_matrix = apply_congestion_1(adjacency_matrix, n1, n2, n3)
    print(f"Running Dijkstra's Algorithm from {start_node} to {end_node}")
    distance, path = dijkstra(adjacency_matrix, start_node, end_node, num_nodes)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    # Run Bellman-Ford Algorithm after congestion scenario 1
    print(f"Running Bellman-Ford Algorithm from {start_node} to {end_node}")
    distance, path = bellman_ford(adjacency_matrix, num_nodes, start_node, end_node)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    edges_1 = adjacency_matrix_to_edge_list(adjacency_matrix)

    # Apply congestion scenario 2 and run Dijkstra
    print("Applying Congestion Scenario 2...")
    adjacency_matrix = apply_congestion_2(adjacency_matrix, n1, n2, n3)
    print(f"Running Dijkstra's Algorithm from {start_node} to {end_node}")
    distance, path = dijkstra(adjacency_matrix, start_node, end_node, num_nodes)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    # Run Bellman-Ford Algorithm after congestion scenario 2
    print(f"Running Bellman-Ford Algorithm from {start_node} to {end_node}")
    distance, path = bellman_ford(adjacency_matrix, num_nodes, start_node, end_node)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    edges_2 = adjacency_matrix_to_edge_list(adjacency_matrix)
    
    # Apply congestion scenario 3 and run Dijkstra
    print("Applying Congestion Scenario 3...")
    adjacency_matrix = apply_congestion_3(adjacency_matrix, n1, n2, n3)
    print(f"Running Dijkstra's Algorithm from {start_node} to {end_node}")
    distance, path = dijkstra(adjacency_matrix, start_node, end_node, num_nodes)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    # Run Bellman-Ford Algorithm after congestion scenario 3
    print(f"Running Bellman-Ford Algorithm from {start_node} to {end_node}")
    distance, path = bellman_ford(adjacency_matrix, num_nodes, start_node, end_node)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")

    # Convert the modified adjacency matrix back to edge list
    edges_3 = adjacency_matrix_to_edge_list(adjacency_matrix)

    # Running Dijkstra's Algorithm for shortest path with the modified network
    print(f"Running Dijkstra's Algorithm with modified network from {start_node} to {end_node}")
    distance, path = dijkstra(adjacency_matrix, start_node, end_node, num_nodes)
    print(f"Shortest distance from node {start_node} to node {end_node} is {distance}")
    print(f"Path: {path}")


    
     # Initialize the networks and explicitly add nodes
    network = Network(notebook=True, cdn_resources="remote")
    network_1 = Network(notebook=True, cdn_resources="remote")
    network_2 = Network(notebook=True, cdn_resources="remote")
    network_3 = Network(notebook=True, cdn_resources="remote")
    

    network.set_options("""
    var options = {
        "physics": {
                        
        "enabled": false
        }
    }
    """)

    network_1.set_options("""
    var options = {
        "physics": {
            "enabled": false
        }
    }
    """)
    
    network_2.set_options("""
    var options = {
        "physics": {
            "enabled": false
        }
    }
    """)
    network_3.set_options("""
    var options = {
        "physics": {
            "enabled": false
        }
    }
    """)
    for i in range(12):
            network.add_node(i, label=f"PC{i}")
            network_1.add_node(i, label=f"PC{i}")
            network_2.add_node(i, label=f"PC{i}")
            network_3.add_node(i, label=f"PC{i}")
   
    color_switch ="#808080"  
    for i in range(6):
            network.add_node(i+12, label=f"S{i}",color=color_switch)
            network_1.add_node(i+12, label=f"S{i}",color=color_switch)
            network_2.add_node(i+12, label=f"S{i}",color=color_switch)
            network_3.add_node(i+12, label=f"S{i}",color=color_switch)
            
    
    color_router = "#000000"
    for i in range(5):
            network.add_node(i+18, label=f"R{i}",color=color_router)
            network_1.add_node(i+18, label=f"R{i}",color=color_router)
            network_2.add_node(i+18, label=f"R{i}",color=color_router)
            network_3.add_node(i+18, label=f"R{i}",color=color_router)
   
            
    for i in range(len(edges)):
         network.add_edge(edges[i][0], edges[i][1], label=str(edges[i][2]), title=f"Weight: {edges[i][2]}")
         network_1.add_edge(edges_1[i][0], edges_1[i][1], label=str(edges_1[i][2]), title=f"Weight: {edges_1[i][2]}")
         network_2.add_edge(edges_2[i][0], edges_2[i][1], label=str(edges_2[i][2]), title=f"Weight: {edges_2[i][2]}")
         network_3.add_edge(edges_2[i][0], edges_3[i][1], label=str(edges_3[i][2]), title=f"Weight: {edges_3[i][2]}")

    # Visualize networks
    network.show("node.html")
    network_1.show("node1.html")
    network_2.show("node2.html")
    network_3.show("node3.html")


# Call the main function
if __name__ == "__main__":
    main()


Generated Edges: [(12, 0, 3), (12, 1, 3), (13, 2, 3), (13, 3, 3), (14, 4, 3), (14, 5, 3), (15, 6, 3), (15, 7, 3), (16, 8, 3), (16, 9, 3), (17, 10, 3), (17, 11, 3), (12, 18, 3), (13, 18, 3), (14, 18, 3), (15, 19, 3), (16, 19, 3), (17, 19, 3), (20, 14, 3), (20, 15, 3), (20, 16, 3), (18, 19, 3), (18, 20, 3), (19, 20, 3), (21, 22, 3), (20, 22, 3), (19, 22, 3), (18, 21, 3), (19, 21, 3)]
Adjacency Matrix:
[[ 0. inf inf inf inf inf inf inf inf inf inf inf  3. inf inf inf inf inf
  inf inf inf inf inf]
 [inf  0. inf inf inf inf inf inf inf inf inf inf  3. inf inf inf inf inf
  inf inf inf inf inf]
 [inf inf  0. inf inf inf inf inf inf inf inf inf inf  3. inf inf inf inf
  inf inf inf inf inf]
 [inf inf inf  0. inf inf inf inf inf inf inf inf inf  3. inf inf inf inf
  inf inf inf inf inf]
 [inf inf inf inf  0. inf inf inf inf inf inf inf inf inf  3. inf inf inf
  inf inf inf inf inf]
 [inf inf inf inf inf  0. inf inf inf inf inf inf inf inf  3. inf inf inf
  inf inf inf inf inf]
 [inf inf inf i